# Explorative Datenanalyse eines Online-Shops


In [ ]:
# Bibliotheken importieren
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os



In [ ]:
# CSV-Datei laden
datafile = os.path.join("..", "data", "data.csv")
df = pd.read_csv(datafile, encoding="latin1", parse_dates=["InvoiceDate"])

# Neue Spalte 'TotalPrice' berechnen
df["TotalPrice"] = df["Quantity"] * df["UnitPrice"]

In [ ]:
# Überblick über die Struktur und ersten Zeilen der Daten
df.info()
df.head()


In [ ]:
# Statistische Übersicht der numerischen Spalten
df.describe()

In [ ]:
# Gesamtumsatz
total_revenue = df["TotalPrice"].sum()
print(f"Gesamtumsatz: {total_revenue:.1f} GBP")

# Durchschnittlicher Warenkorbwert
average_order_value = df.groupby("InvoiceNo")["TotalPrice"].sum().mean()
print(f"Durchschnittlicher Warenkorbwert: {average_order_value:.1f} GBP")

In [ ]:
# Umsatz pro Monat visualisieren
monthly_revenue = df.set_index("InvoiceDate").resample("M")["TotalPrice"].sum()

plt.figure(figsize=(10,6))
monthly_revenue.plot()
plt.title("Umsatz pro Monat")
plt.xlabel("Datum")
plt.ylabel("Umsatz (GBP)")
plt.tight_layout()
plt.show()


##  Produktanalyse
Untersuchung der meistverkauften Produkte

In [ ]:


# Top 10 Produkte nach Verkaufsmenge
top_products = df.groupby("Description")["Quantity"].sum().sort_values(ascending=False).head(10)

plt.figure(figsize=(10,6))
sns.barplot(x=top_products.values, y=top_products.index)
plt.title("Top 10 Produkte nach Verkaufsmenge")
plt.xlabel("Anzahl verkauft")
plt.ylabel("Produkt")
plt.tight_layout()
plt.show()



## Umsatzanalyse
Analyse des Gesamtumsatzes und der monatlichen Entwicklung


In [ ]:
# Umsatz nach Monat
df["Month"] = df["InvoiceDate"].dt.to_period("M")
monthly_sales = df.groupby("Month")["TotalPrice"].sum()

monthly_sales.plot(kind="bar", figsize=(12,5), title="Umsatz pro Monat")
plt.ylabel("Umsatz in GBP")
plt.tight_layout()
plt.show()



## Kundenanalyse
Analyse der Top-Kunden und ihres Kaufverhaltens


In [ ]:
top_customers = df.groupby("CustomerID")["TotalPrice"].sum().sort_values(ascending=False).head(10)
top_customers.plot(kind="bar", title="Top 10 Kunden nach Umsatz")
plt.xlabel("Kunden-ID")
plt.ylabel("Umsatz")
plt.tight_layout()
plt.show()


## Verbesserung: Datenbereinigung
Bevor mit der Analyse begonnen wird, sollten die Daten bereinigt werden (z.B. fehlende Werte, Dubletten, negative Mengen/Preise, Stornierungen).

In [ ]:
# Fehlende Werte prüfen
print(df.isnull().sum())

# Dubletten entfernen
df = df.drop_duplicates()

# Negative Mengen und Preise entfernen (Stornierungen)
df = df[(df["Quantity"] > 0) & (df["UnitPrice"] > 0)]

# Nach der Bereinigung Überblick verschaffen
df.info()

## Erweiterung: RFM-Analyse (Recency, Frequency, Monetary)
Segmentierung der Kunden nach Kaufverhalten.

In [ ]:
import datetime

snapshot_date = df["InvoiceDate"].max() + pd.Timedelta(days=1)
rfm = df.groupby("CustomerID").agg({
    "InvoiceDate": lambda x: (snapshot_date - x.max()).days,
    "InvoiceNo": "nunique",
    "TotalPrice": "sum"
})
rfm.rename(columns={"InvoiceDate": "Recency", "InvoiceNo": "Frequency", "TotalPrice": "Monetary"}, inplace=True)
rfm.head()